# Logging 

# Why logging?

* Debugging "after the fact".
* Provides a trace of event that lead to a certain cituation.
* Logs can be archived and compared.
* Logs can be monitord during production.
* Plenty of information with minimal effort (level, timestamp).

# Console logging

In [1]:
# %load examples/logconsole.py
import logging

logging.basicConfig(level=logging.INFO)

log = logging.getLogger('myapp')
log.info('Hello world')

name = 'Alice'
size = 172
log.info('%s is %dcm tall', name, size)

Output:
```
INFO:myapp:Hello world
INFO:myapp:Alice is 172cm tall
```

## Logging level

* INFO = normal information about what the program is doing / has done; should use terms known to the end user
* DEBUG = additional information that might only be useful to the developer; can refer to internal variable names etc.
* ERROR = details about a step that could not be performed
* CRITICAL = details about critical errors; this is mostly useful for services, where ERROR can be recovered from and CRITICAL causes the service to shut down
* WARNING: information about error that could be corrected, e.g. by provididing a default

## Logging messages

* Loggers have methods to log messages for each level: `Logger.info()`, `Logger.error()` etc.
* `Logger.exception()` is similar to `Logger.error()` but also shows the stack trace.
* Messages can be a string or a format pattern with variables to log separated by a comma.
* Message formats only support classic placeholders like `%s` and `%d`, but not the newer `%(name)s` or `{0}` formats.

## Logging messages (continued)

Compute the sum of numers stored in a text file:

In [2]:
log = logging.getLogger('numbers')
result = 0
numbers_path = os.path.join('examples', 'numbers.txt')
log.info('read numbers from %s', numbers_path)
line_number = 0
try:
    with open(numbers_path, 'r', encoding='utf-8') as numbers_file:
        for line_number, line in enumerate(numbers_file, 1):
            number_text = line.rstrip('\n\r')
            if number_text != '':
                log.debug('  %d: %s', line_number, number_text)
                result += int(number_text)
    log.info('sum = %d', result)
except FileNotFoundError:
    log.warning('cannot find numbers file %s, using default %d', result)
except OSError as error:
    log.error(error)
except TypeError as error:
    log.error(
        '%s:%d: line must be a number but is: %s',
        config_path, line_number, number_text)
except Exception as error:
    log.exception(error)  # show stack trace for unexpected errors

NameError: name 'os' is not defined

## Logging performance

What is the difference between

In [ ]:
log.info('reading config from %s', config_path)

and

In [ ]:
log.info('reading config from %s' % config_path)

Anwer: the latter must resolve the formatted string even if the messages does not need to be logged. This can reduce performace, especially when using `Logger.debug()`.

## Logging handlers

* The [logging.handlers](https://docs.python.org/3/library/logging.handlers.html#module-logging.handlers) package provides many useful standard handlers, for example:
  * `FileHandler` - logs to a file
  * `RotatingFileHandler` - logs multiple sessions to a file; large files are rotated
  * `SocketHandler` - logs to a network socket
  * `HTTPHandler` - sends log messages to a HTTP server using `GET` or `POST`
* You can also implement your own handlers.
* Messages can be logged to multiple handlers at the same time.

## Log message formatters

Formatters specify, what additional information is logged with the actual message. Among others, this can include:
* A timestamp: `%(asctime)s` 
* The log level: `%(levelname)s`
* The name of the logger: `%(name)s`
* The actual message: `%(message)s`

# Logging configuration

Logging can be configured using several ways, for example:
* `logging.basicConfig()`, which simply logs to the console (`stderr`); very useful for little command line tools
* Programatically using `addHandler()` etc.
* Using dictionaries and `logging.config.dictConfig()`.
* Using configuration files and `logging.config.fileConfig()`.

## Logging configuration files

* Simple config files that can be parser using `configparser.ConfigParser`
* Sections to include: `[logging]`, `[handlers]`, `[formatters]`
* Separates code and logging configuration
* Can be changed by non developers (e.g. operations)
* No need to rebuild and redeploy the package

## Example logging configuration (1/3)

```ini
# Logging
[loggers]
keys=root, myapp

[handlers]
keys=console, file

[formatters]
keys=standard

[logger_root]
level=NOTSET
handlers=

[logger_myapp]
level=DEBUG
handlers=console, file
propagate=1
qualname=myapp
```

## Example logging configuration (2/3)

```ini
[handler_console]
class=StreamHandler
level=INFO
formatter=standard
args=(sys.stdout,)

[handler_file]
class=FileHandler
level=DEBUG
formatter=standard
args=(r'/tmp/myapp.log', 'w')

[formatter_standard]
format=%(asctime)s %(levelname)s %(message)s
datefmt=
class=logging.Formatter
```

## Example logging configuration (3/3)

* Logger `myapp` is qualified and works for e.g. `myapp`, `myapp.some`, `myapp.other` etc.
* Logger `root` handles everything else and just does nothing.
* Handler `console` logs to `stderr`, but only level `INFO` or higher.
* Handler `file` logs to a log file starting with level `DEBUG`.
* Both handlers use a formatter `standard` that includes a timestamp.

## Example application

In [3]:
# %load examples/myapp.py
# Simple example application reading a config.
import configparser
import logging
import logging.config
import logging.handlers
import os.path

log = logging.getLogger('myapp')


class MyApp(object):
    def __init__(self, config_path):
        # Setup config with defaults.
        config = configparser.ConfigParser()
        config['myapp'] = {}
        config['myapp']['fullscreen'] = 'false'
        config['myapp']['timeout'] = '60'

        # Update config from file.
        with open(config_path, 'r', encoding='utf-8') as config_file:
            config.read_file(config_file)

        self.connection = config.get('myapp', 'connection')
        self.is_fullscreen = config.getboolean('myapp', 'fullscreen')
        self.timeout = config.getint('myapp', 'timeout')

    def work(self):
        log.debug('connection=%s', self.connection)
        log.info('is_fullscreen=%s', self.is_fullscreen)
        log.info('timeout=%d', self.timeout)


if __name__ == '__main__':
    config_path = os.path.join('examples', 'myapp.cfg')
    logging.config.fileConfig(config_path)
    myapp = MyApp(config_path)
    myapp.work()


2016-03-10 17:42:48,491 INFO is_fullscreen=True
2016-03-10 17:42:48,492 INFO timeout=10


# Summary

* Logging can be helpful during error analysis to see what happened after the fact.
* Python provides a flexible `logging` module.
* Messages have a level (`INFO`, `ERROR`, ...)
* Messages can be logged to one or more handler.
* There are standard handlers to log to files and the network.
* Logging can be configured programatically or using configuration files.